**Business Goal :**  To develop an AI-powered chatbot (Vajra.AI) for customer interaction and improve service delivery at Social Prachar Institute.

**Purpose :**
*   Assist students with queries related to courses, fees, institute details, internships, and more.
*   Provide 24/7 customer support without the need for human intervention.
*   Reduced manual workload for staff handling repetitive inquiries.
*   Increased customer satisfaction through quick and accurate responses.:






**Installing necessary libraries**

In [11]:
!pip install nltk
!pip install tensorflow
!pip install numpy

In [12]:
import pandas as pd
import numpy as np
import json
import random
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


**Defining User Intent and Response Patterns for Chatbot**

In [13]:
# Initialize Lemmatizer
lemmatizer = nltk.WordNetLemmatizer()

# Example training data (same as provided)
intents = [
    {
        "intent": "greetings",
        "patterns": ["Hello", "Hi", "Good day", "Hey", "How are you?", "Good morning", "Hi there"],
        "responses": [
            "Hello! Welcome to Social Prachar Institute. How can I assist you today?",
            "Hi there! How can I help you with your queries about our courses?",
            "Good day! I’m Vajra.AI, your guide to the Social Prachar Institute. What would you like to know?"
        ]
    },
    {
        "intent": "farewells",
        "patterns": ["Goodbye", "Bye", "Thank you", "See you later", "Take care", "Farewell"],
        "responses": [
            "Thank you for visiting! Have a great day!",
            "It was my pleasure to assist you. Take care and good luck with your learning journey!",
            "Goodbye, and feel free to come back if you have more questions!"
        ]
    },
    {
        "intent": "institute_details",
        "patterns": ["What is the name of the institute?", "Where are you located?", "Tell me about your institute?", "What is Social Prachar Institute?"],
        "responses": [
            "The name of the institute is Social Prachar.",
            "Social Prachar is an educational institute offering courses in Data Science & AI, Data Analytics, Python Full Stack, Java Full Stack, and AWS Developer. We focus on providing high-quality training to our students with a strong emphasis on hands-on experience and practical knowledge.",
            "We are located at [hi-tech city road, 216, Manjeera Majestic Commercial, JNTU Rd, near JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY, Hyderabad, Telangana 500085]."
        ]
    },
    {
        "intent": "course_info",
        "patterns": ["What courses do you offer?", "Tell me about the courses", "What courses are available?", "Can you list your courses?"],
        "responses": [
            "We offer Data Science, Data Analytics, Python Full Stack, Java Full Stack, and AWS Developer courses.",
            "Our courses include Data Science & AI, Data Analytics, Python Full Stack, Java Full Stack, and AWS Developer. You can learn more about each course on our website."
        ]
    },
    {
        "intent": "fee_structure",
        "patterns": ["How much does the course cost?", "What are the fees for the courses?", "Tell me the course fees."],
        "responses": [
            "The fee for Data Science is 50k, and other courses are 30k.",
            "Our courses are priced as follows: Data Science - 50k, Data Analytics - 30k, Python Full Stack - 30k, Java Full Stack - 30k, AWS Developer - 30k."
        ]
    },
    {
        "intent": "internship_opportunity",
        "patterns": ["Do you offer internships?", "Are there internship opportunities?", "Can I get an internship after completing the course?"],
        "responses": [
            "Yes, internships are available for some courses like Data Science and Python Full Stack.",
            "We offer internships for selected courses such as Data Science and Python Full Stack. Internships are based on performance and course requirements."
        ]
    },
    {
        "intent": "placement_assistance",
        "patterns": ["Do you provide placement assistance?", "Will you help me get a job?", "Can you help me with jobs?"],
        "responses": [
            "Yes, we provide placement assistance, including resume building and interview preparation.",
            "We offer placement assistance services such as resume building, mock interviews, job placement, and referrals."
        ]
    },
    {
        "intent": "refund_policy",
        "patterns": ["What is your refund policy?", "Can I get a refund if I don't like the course?", "Do you have a refund policy for your courses?"],
        "responses": [
            "We offer a 7-day refund policy if you're not satisfied with the course.",
            "If you're not satisfied with the course, we offer a 7-day refund policy."
        ]
    },
    {
        "intent": "demo_classes",
        "patterns": ["Do you offer demo classes?", "Can I try a demo class?", "Where can I book a demo class?"],
        "responses": [
            "You can book your demo class by visiting [Social Prachar Institute](https://socialprachar.com/). Demo classes are available for all courses.",
            "Demo classes are available for all courses. You can book your demo class on our website."
        ]
    },
    {
        "intent": "payment_methods",
        "patterns": ["What are the payment options?", "What payment methods do you accept?", "Can I pay online for the courses?"],
        "responses": [
            "We accept all types of payments.",
            "You can pay via credit card, debit card, online banking, or other accepted methods. All payment options are listed on our website."
        ]
    },
    {
        "intent": "error_handling",
        "patterns": ["I didn't understand.", "I don't know what you mean.", "Can you explain it again?"],
        "responses": [
            "Sorry, I didn’t quite get that. Could you please rephrase your question?",
            "I didn't quite understand. Could you please rephrase it?"
        ]
    },
    {
        "intent": "spelling_mistake",
        "patterns": ["I think I made a spelling mistake.", "I might have typed it wrong.", "Could you check for spelling mistakes?"],
        "responses": [
            "I noticed a spelling mistake in your question. Could you please check and rephrase it?",
            "There seems to be a spelling mistake. Could you please check and correct it?"
        ]
    },
    {
        "intent": "institute_name",
        "patterns": ["What is the name of the institute?", "Which institute is this?", "Tell me the institute name", "What is the name of your institute?"],
        "responses": [
            "The name of the institute is Social Prachar."
        ]
    }
]



**Data Preprocessing**

In [14]:
# Preprocessing: Tokenization and Lemmatization
def preprocess_sentence(sentence):
    sentence_tokens = nltk.word_tokenize(sentence.lower())  # Tokenize
    sentence_tokens = [lemmatizer.lemmatize(word) for word in sentence_tokens if word not in ignore_words]
    return " ".join(sentence_tokens)

# Prepare data
training_sentences = []
training_labels = []
class_names = []
words = []
ignore_words = ['?', '!', '.', ',']
# Data preprocessing
for intent in intents:
    for pattern in intent['patterns']:
        processed_sentence = preprocess_sentence(pattern)  # Apply preprocessing
        training_sentences.append(processed_sentence)  # Use preprocessed sentence
        training_labels.append(intent['intent'])

        # Tokenizing and collecting all words for later vectorization
        word_list = nltk.word_tokenize(pattern)  # Tokenize the sentence
        words.extend(word_list)

    if intent['intent'] not in class_names:
        class_names.append(intent['intent'])

# Lemmatize and remove duplicates from all the words
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))  # Remove duplicates and sort words

# Encode the labels (intent labels to numeric values)
# label_encoder = LabelEncoder()
# labels = label_encoder.fit_transform(training_labels)

# Using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(training_sentences).toarray()

# # Convert labels to categorical format
# y = training_labels

X = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
y = pd.DataFrame(training_labels, columns=['intent'])

**Model Training and Evaluation with Gaussian Naive Bayes**

In [15]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize the Gaussian Naive Bayes model
nb_model = GaussianNB()

# Train the model
nb_model.fit(X_train, y_train)

# Make predictions
y_pred = nb_model.predict(X_test)

print(y_pred)

# Evaluate the model
# Evaluate the model by comparing the predicted and true labels directly (no need for np.argmax)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

['institute_name' 'institute_name' 'institute_details' 'spelling_mistake'
 'internship_opportunity']
Accuracy: 0.400


**Predicting Intent and Providing Responses**

In [17]:
def predict_intent(user_input):
    # Tokenize and lemmatize the user input
    user_input_tokens = nltk.word_tokenize(user_input.lower())
    user_input_tokens = [lemmatizer.lemmatize(word) for word in user_input_tokens if word not in ignore_words]

    # Convert the user input into the same numerical vector using the vectorizer
    input_vector = vectorizer.transform([" ".join(user_input_tokens)]).toarray()

    # Predict the intent using the trained Naive Bayes model
    prediction = nb_model.predict(input_vector)[0]

    print("Intent:",prediction)

    # Rule-based responses for specific intents
    for i in intents:
        if i['intent'] == prediction:
            return random.choice(i['responses'])

    # Default response
    return "Sorry, I didn’t understand that. Could you please rephrase?"

# Main loop for interaction
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Bye! See you again.")
        break
    response = predict_intent(user_input)
    print(f"Vajra.AI: {response}")

You: Hello
Intent: greetings
Vajra.AI: Hi there! How can I help you with your queries about our courses?
You: what is the institute name
Intent: institute_name
Vajra.AI: The name of the institute is Social Prachar.
You: where is the institute located
Intent: institute_details
Vajra.AI: The name of the institute is Social Prachar.
You: where is the institute locate
Intent: institute_details
Vajra.AI: We are located at [hi-tech city road, 216, Manjeera Majestic Commercial, JNTU Rd, near JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY, Hyderabad, Telangana 500085].
You: what are the courses you are offering
Intent: course_info
Vajra.AI: We offer Data Science, Data Analytics, Python Full Stack, Java Full Stack, and AWS Developer courses.
You: data science fee
Intent: fee_structure
Vajra.AI: The fee for Data Science is 50k, and other courses are 30k.
You: demo classes
Intent: demo_classes
Vajra.AI: Demo classes are available for all courses. You can book your demo class on our website.
You: inter

**saving the model**

In [ ]:
import pickle

# Example: Assuming 'model' is a trained machine learning model
model = nb_model  # Your trained model here

# Save the model to a .pkl file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)


**saving the vextorizer file**

In [ ]:
import pickle

# Save the vectorizer to a .pkl file
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
